In [12]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications import VGG16, MobileNetV2, InceptionV3, NASNetLarge
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from efficientnet import EfficientNetB3, center_crop_and_resize, preprocess_input

from keras import models
from keras import layers
from keras import callbacks

import time

import matplotlib.pyplot as plt

import cv2

import pickle

In [6]:
BASEPATH = "../stanford-dogs-dataset/"
BATCH_SIZE = 32
EPOCHS = 100

In [3]:
early_stopping = EarlyStopping(patience=20, verbose=1,restore_best_weights=True, monitor="val_acc")
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=7,verbose=1, monitor="val_acc")

In [4]:
def createModelNASNetLarge(trainable=False, optimizer="adam"):
    base_model = NASNetLarge(weights='imagenet', include_top=False, input_shape=(331, 331, 3))

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(512,activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(120, activation="softmax"))
    
    if not trainable:
        for layer in base_model.layers:
            layer.trainable = False

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) 
  
    return model

In [5]:
model = createModelNASNetLarge()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Using the ImageDataGenerator

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255.0,
                                   shear_range=0.1,
                                   rotation_range=10.,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=[0.9, 1.1],
                                   brightness_range=[0.8, 1.2],
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255.0)

validation_datagen = ImageDataGenerator(rescale=1./255.0)

In [7]:
train_generator = train_datagen.flow_from_directory(
    BASEPATH + "train",
    target_size=(331, 331),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    BASEPATH + "test",
    target_size=(331, 331),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    BASEPATH + "val",
    target_size=(331, 331),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 15394 images belonging to 120 classes.
Found 1114 images belonging to 120 classes.
Found 4072 images belonging to 120 classes.


In [11]:
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // BATCH_SIZE,
    epochs = EPOCHS, callbacks=[early_stopping, reduce_lr])

Epoch 1/100
482/482 [==============================] - 562s 1s/step - loss: 0.3874 - acc: 0.8957 - val_loss: 0.2641 - val_acc: 0.9280
Epoch 2/100
482/482 [==============================] - 561s 1s/step - loss: 0.3390 - acc: 0.9043 - val_loss: 0.2684 - val_acc: 0.9241
Epoch 3/100
482/482 [==============================] - 560s 1s/step - loss: 0.3247 - acc: 0.9087 - val_loss: 0.2726 - val_acc: 0.9268
Epoch 4/100
482/482 [==============================] - 561s 1s/step - loss: 0.3165 - acc: 0.9102 - val_loss: 0.2608 - val_acc: 0.9315
Epoch 5/100
482/482 [==============================] - 561s 1s/step - loss: 0.3013 - acc: 0.9125 - val_loss: 0.2735 - val_acc: 0.9303
Epoch 6/100
482/482 [==============================] - 562s 1s/step - loss: 0.2929 - acc: 0.9166 - val_loss: 0.2660 - val_acc: 0.9285
Epoch 7/100
482/482 [==============================] - 560s 1s/step - loss: 0.2795 - acc: 0.9152 - val_loss: 0.2828 - val_acc: 0.9261
Epoch 8/100
482/482 [==============================] - 560s 1s

Epoch 37/100
128/128 [==============================] - 115s 896ms/step - loss: 0.2693 - acc: 0.9344
Restoring model weights from the end of the best epoch.
482/482 [==============================] - 566s 1s/step - loss: 0.1339 - acc: 0.9563 - val_loss: 0.2693 - val_acc: 0.9344
Epoch 00037: early stopping


## Evaluation

In [9]:
model = load_model("../models/nasnetlarge-model-0.94acc-0.28loss-1561191444.hdf5")

W0629 16:17:17.050178 4568139200 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0629 16:17:17.051109 4568139200 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0629 16:17:17.053171 4568139200 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will b

In [11]:
loss, acc = model.evaluate_generator(test_generator,verbose=1, steps=test_generator.samples // BATCH_SIZE)

13/34 [==========>...................] - ETA: 4:56 - loss: 0.1976 - acc: 0.9447

KeyboardInterrupt: 

In [ ]:
print(loss,acc)

In [14]:
model.save("nasnetlarge-model-{:.2f}acc-{:.2f}loss-{:.0f}.hdf5".format(acc, loss, time.time()))